In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import cycle, islice
from IPython.display import clear_output
import scipy
from scipy import stats
import operator
from functools import reduce
import datetime as dt
import time
import math
import csv
import sys
import PIL
import os
__array_ufunc__ = None

## Data sources

In [ ]:
file_list=[]
stats_file_list=[]

for subdir, dirs, files in os.walk("C:\\Users\\data"):
    for file in files:
        filepath = subdir + os.sep + file
        if file.endswith(".csv"):
            if file.endswith("stats.csv"):
                stats_file_list.append(filepath)
            else:
                file_list.append(filepath)

print("Found", len(file_list), "radiance files and", len(stats_file_list), "stats files")
assert len(file_list) == len(stats_file_list), "number of radiance files and stats files don't match"

preview_file = pd.read_csv(file_list[0])
preview_file['period'] = pd.to_datetime(preview_file['period']) 
deltaT = preview_file['period'].max()-preview_file['period'].min()
year_count = int(round(deltaT.days/365))
print("Found", year_count, "years of data in first file")

all_columns = preview_file.columns
lat_bins = all_columns[2:]
lat_bins = lat_bins[::-1]
print("First file has these", len(lat_bins), "latitude bins:")
print(lat_bins)
print("The other files must have the same bins")
preview_file['wavenumber'] = preview_file['wavenumber'].round(3) # sometimes source data has too many decimal places
wavenums = preview_file['wavenumber'].round(3).unique()  # Get all of the unique wavenumbers

## Set Parameters

In [ ]:
# Minimum number of measurements for trend fitting.
# If OLRdata_stats has fewer than MIN_COUNT measurements in it for a given month,
# then the trendlines shall ignore that month for the sake of fitting a trendline. 
MIN_COUNT = 0

# Minimum number of months needed to infer a linear trend from each wavenumber+latitude time series
# Setting this number lower includes channels that may have failed after weeks or months
# Setting this number higher excludes proportionally more thannels that failed or failed and were restored later
MIN_MONTHS_TO_OBTAIN_SLOPE = 48

# AIRS stability is characterized by others (Strow 2020, Aumann 2016, Pagano) using surface reference temperatures.
# Enter the degrees kelvin per year the instrument drifts:
drift_K = 0.0023       #Longwave <1700 cm^-1 
drift_K2 = 0.008     #Midwave >2100 cm^-1

# CO2 band definitions
v2start = 649.620-0.1
v2end = 681.729+0.1
v2wingstart = 687.601-0.1
v2wingend = 764.534+0.1
v3start = 2195.158
v3end = 2395.995
win_start = 791.035
win_end = 792.814
interpolate_start = 681.7

### Progress bar function (helpful in next section, it is slow depending on amount of data processed)

In [ ]:
def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.3f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

## Filter data and fit linear regression

In [ ]:
total_masked = total_count = filtered_count = 0
i, total_actions = 0, (2*2378*len(lat_bins)+3*len(lat_bins))*len(file_list)
all_radiances = {}
all_radiance_trends = {}
stat_columns = ['cell1', 'cell2', 'cell3', 'cell4']
statsdf = pd.DataFrame(index=file_list, columns=stat_columns)
statsdf.fillna(0,inplace=True)
statsdf2 = pd.DataFrame()

for file in file_list:
    OLRdata = pd.read_csv(file)
    OLRdata['wavenumber'] = OLRdata['wavenumber'].round(3) # sometimes source data has too many decimal places
    OLRdata_stats = pd.read_csv(file.rsplit('.', 1)[0]+"_stats.csv") # read in stats file for these OLR data
    OLRdata_stats.columns = OLRdata.columns # overwrite the columns in the stats file to be identical to OLR data
    OLRdata_stats['wavenumber'] = OLRdata_stats['wavenumber'].round(3) # sometimes source data has too many decimal places
    
    statsdf["cell1"][file]=OLRdata_stats.iloc[:, 2:].count().sum() # Store baseline statistics "as-read"
    
    # Mask data where no measurement exists: convert any data whose corresponding count is 0 to NaN
    for column in lat_bins:
        i+=1; update_progress(i / total_actions)
        OLRdata[column] = OLRdata[column].mask(OLRdata_stats[column] < 1, np.nan)
        OLRdata_stats[column]= OLRdata_stats[column].mask(OLRdata_stats[column] < 1, np.nan)
    
    statsdf["cell2"][file]=OLRdata_stats.iloc[:, 2:].count().sum() # Store statistics after filtering 0's
    
    # Convert any data whose corresponding count is less than MIN_COUNT to NaN
    if MIN_COUNT > 0:
        for column in lat_bins:
            i+=1; update_progress(i / total_actions)
            OLRdata[column] = OLRdata[column].mask(OLRdata_stats[column] < MIN_COUNT, np.nan)
            OLRdata_stats[column]= OLRdata_stats[column].mask(OLRdata_stats[column] < MIN_COUNT, np.nan)
    statsdf["cell3"][file]=OLRdata_stats.iloc[:, 2:].count().sum() # Store statistics after filtering <MIN_COUNT
    
    # Mask channels with fewer months than user minimum criteria
    if MIN_MONTHS_TO_OBTAIN_SLOPE > 0:
        for column in lat_bins:
            col_series = OLRdata[column]
            mask_series = pd.Series(False, index=OLRdata.index) # to keep track of items to be removed
            for wn in wavenums:
                i+=1; update_progress(i / total_actions)
                timeseries = col_series[OLRdata['wavenumber'] == wn]
                if (~np.isnan(timeseries)).sum() < MIN_MONTHS_TO_OBTAIN_SLOPE:   # If total non-NAN points is insufficient
                    mask_series = (mask_series | (OLRdata['wavenumber'] == wn))  # add the matching points to our mask
            total_masked += mask_series.sum()
            OLRdata[column] = OLRdata[column].mask(mask_series, np.nan)
            OLRdata_stats[column] = OLRdata_stats[column].mask(mask_series, np.nan)
    statsdf["cell4"][file]=OLRdata_stats.iloc[:, 2:].count().sum() # Store count after filter <MIN_MONTHS_TO_OBTAIN
    statsdf2=statsdf2.append(OLRdata_stats)  # Append aggregate filtered states for later median determination
    
    all_radiances[file]={}
    all_radiance_trends[file]={}
    for lat_index, latitude in enumerate(lat_bins): # This is an enumerate call, so that we can index 
        df3 = OLRdata.set_index(['period', 'wavenumber'])[latitude].unstack() # Re-arrange so each wavenum in its own column
        df3.index = pd.DatetimeIndex(df3.index) # Re-format the index's date strings into numerical dates
        linmodels_dict = {} # Fitting linear models
        dates = df3.index # Need to convert the dates into months in order to perform regression
        mo = dates.year * 12 + dates.month
        for col in df3.columns:
            i+=1; update_progress(i / total_actions)
            ser = df3[col] # Get the individual series
            d = {}
            varx = mo
            vary = ser.values
            mask = ~np.isnan(varx) & ~np.isnan(vary)
            if sum(mask) > 0: # Perform linear regression
                d['Slope'], d['Intercept'], d['R'], d['P-value'], d['Std. Err'] = stats.linregress(varx[mask], vary[mask])
                d['avg'] = vary[mask].mean()
                linmodels_dict[col] = d
        linmodels_df = pd.DataFrame(linmodels_dict).transpose()  # Compile a DataFrame of the linear models, and save it
        linmodels_df["R^2"] = linmodels_df["R"] ** 2     # Get R^2 for additional information
        linmodels_df["Slope"] = linmodels_df["Slope"] * 12    # Convert slope monthly -> yearly
        all_radiances[file][latitude] = pd.DataFrame(df3)
        all_radiance_trends[file][latitude] = {
            "wavenumbers": linmodels_df.index,
            "slope": linmodels_df["Slope"],
            "avg": linmodels_df['avg'],
        }
update_progress(1)

In [ ]:
print("Percent of grid cells with no data:", '{:.2f}'.format((1-(statsdf['cell2'].sum()/statsdf['cell1'].sum()))*100), '%')
if MIN_COUNT > 0:
    print("Grid cells <MIN_COUNT:", '{:.2f}'.format((1-(statsdf['cell3'].sum()/statsdf['cell2'].sum()))*100), '%')
    print("Grid cells <MIN_MONTHS:", '{:.2f}'.format((1-(statsdf['cell4'].sum()/statsdf['cell3'].sum()))*100), '%')
else:
    print("Grid cells <MIN_MONTHS:", '{:.2f}'.format((1-(statsdf['cell4'].sum()/statsdf['cell2'].sum()))*100), '%')
print("Median radiance measurements in a grid cell:", statsdf2.iloc[:, 2:].stack().median())
print("Mean radiance measurements in a grid cell:", statsdf2.iloc[:, 2:].stack().mean())
print('Measurement count:', '{:.2f}'.format(statsdf2.iloc[:, 2:].sum(axis=0).sum()/1E9),'billion')
Count_sub_1614=pd.DataFrame()
Count_sub_1614=statsdf2.loc[statsdf2.loc[:,"wavenumber"]<1614]
print('Measurement count <1614 cm-1:', '{:.2f}'.format(Count_sub_1614.iloc[:, 2:].sum(axis=0).sum()/1E9),'billion')
Count_sub_765=pd.DataFrame()
Count_sub_765=statsdf2.loc[statsdf2.loc[:,"wavenumber"]<765]
print('Measurement count <765 cm-1:', '{:.2f}'.format(Count_sub_765.iloc[:, 2:].sum(axis=0).sum()/1E9),'billion')

## Planck Functions

In [ ]:
def fwd_planck(v, T):
    '''
    Receives: wavenumber, blackbody temperature
    Returns: spectral radiance
    equation from https://ncc.nesdis.noaa.gov/data/planck.html
    '''
    return ((0.00001191042*v**3)/(np.exp(1.4387752*v/T)-1))

In [ ]:
def inv_planck(v, L): 
    '''
    Receives: wavenumber, spectral radiance
    Returns: Planck brightness temperature
    equation from https://ncc.nesdis.noaa.gov/data/planck.html
    '''
    return (1.4387752*v/(np.log(0.00001191042*v**3/L+1)))

## Area-weighted averages & error

In [ ]:
# Planet regarded as a sphere, creating area-proportionate weighting factors
lat_deg = np.arange(90, -1, -10).tolist()
lat_rad = np.radians(lat_deg)
lat_sin = np.sin(lat_rad)
lat_dsin = lat_sin[0]-lat_sin
cap_area = 2 * np.pi * lat_dsin
cap_shift = np.insert(cap_area, 0,0)
belt_area = cap_area - cap_shift[:-1]
globe_area = np.concatenate([belt_area[1:], np.flip(belt_area[1:])])
weights=globe_area/np.sum(globe_area)

weights2=[]  # when processing multiple longitude bins, weight the result by # of longitude bins
for latitude_weight in weights:
    weights2.append(latitude_weight/len(file_list))

assert round(sum(weights2)*len(file_list), 6) == 1, "weights don't sum to 1"

In [ ]:
avgR = pd.DataFrame(0, index=wavenums, columns=lat_bins)
series_averagedslopes = pd.Series(index=wavenums, data=0)
lat_rad_err={}
lat_rad_trends={}
for latitude in lat_bins:
    lat_rad_trends[latitude]= pd.Series(index=wavenums, data=0)
    lat_rad_err[latitude] = pd.Series(index=wavenums, data=0)


for lat_index, latitude in enumerate(lat_bins):
    for file in file_list:
        series_averagedslopes += all_radiance_trends[file][latitude]['slope'] * weights2[lat_index]
        avgR[latitude] += all_radiance_trends[file][latitude]['avg'] / len(file_list)
        lat_rad_err[latitude] += all_radiance_trends[file][latitude]['avg'] / len(file_list)
        lat_rad_trends[latitude] += all_radiance_trends[file][latitude]['slope'] / len(file_list)
    lat_rad_err[latitude] = inv_planck(lat_rad_err[latitude].index, lat_rad_err[latitude].values)
    lat_rad_err[latitude] = fwd_planck(wavenums, (lat_rad_err[latitude].values + drift_K)) - fwd_planck(wavenums, lat_rad_err[latitude].values)


avgR = np.sqrt(avgR**2 * weights**2)
avgT = pd.Series(index=wavenums, data=inv_planck(avgR.index, avgR.sum(axis=1).values))
series_averagederror = pd.Series(index=wavenums, data= fwd_planck(avgT.index, (avgT.values + drift_K)) - fwd_planck(avgT.index, avgT.values))
mw_error = pd.Series(index=wavenums, data= fwd_planck(avgT.index, (avgT.values + drift_K2)) - fwd_planck(avgT.index, avgT.values))
mw_error = mw_error.drop(mw_error.index[np.where(mw_error.index < 1800)[0]])
series_averagederror.update(mw_error)

### Process Radiance Trends (slopes)

In [ ]:
slopes_df = pd.DataFrame(index = series_averagedslopes.index, data = {'slope':series_averagedslopes.values})
slopes_df.index.name = 'WN'

# 1. Remove NaNs by linear interpolation
slopes_df1 = slopes_df.copy(deep=True)
slopes_df1 = slopes_df1.interpolate(method = 'linear', limit = 10, limit_area = 'inside')

# 2. Insert 0 value on ordinate where adjacent slopes are opposite sign
slopes_df2 = slopes_df1.copy(deep=True)
for i in range(340, 390, 1):
    if ((slopes_df2['slope'].iloc[i] < 0 and slopes_df2['slope'].iloc[i+1] > 0) or (slopes_df2['slope'].iloc[i] > 0 and slopes_df2['slope'].iloc[i+1] < 0)):
        y1, y2, y3 = slopes_df2['slope'].iloc[i], 0, slopes_df2['slope'].iloc[i+1]
        x1, x3 = slopes_df2.index[i], slopes_df2.index[i+1]
        slopes_df2.loc[(y2-y1)*(x3-x1)/(y3-y1) + x1] = 0
slopes_df2 = slopes_df2.sort_index()

# 3. Remove positive slope values, presuming they are transparent wavenumbers viewing surface heating (not atmosphere)
slopes_df3 = slopes_df2.copy(deep=True)
slopes_df3[slopes_df3 > 0] = 0

slopes_interp = slopes_df[np.isnan(slopes_df['slope'])]  # dataframe of all NaNs
slopes_interp.update(slopes_df1)  # Replace NaNs with actual slopes
slopes_interp.index.name = 'WN'

## $v_2$ wing examination

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10.5,3))       
ax[0].set_title("Raw v2 wing")
ax[0].fill_between(series_averagedslopes[v2wingstart:v2wingend].index,
                   series_averagedslopes[v2wingstart:v2wingend].values, 0, color='grey', linewidth = 0)
ax[1].set_title("Interpolated v2 wing")
ax[1].fill_between(slopes_df2[v2wingstart:v2wingend].index,
                   slopes_df2[v2wingstart:v2wingend]['slope'].values, 0, color='grey', linewidth = 0)

markerline, stemlines, baseline = ax[1].stem(slopes_interp[v2wingstart:v2wingend].index,
                                             slopes_interp[v2wingstart:v2wingend]['slope'].values,
                                             markerfmt=' ', linefmt="k-", basefmt="grey", use_line_collection = True)
plt.setp(stemlines, 'linewidth', 0.25)
plt.setp(baseline, 'linewidth', 0)

ax[2].set_title("Negative-only v2 wing")
ax[2].fill_between(slopes_df3[v2wingstart:v2wingend].index, 
                   slopes_df3[v2wingstart:v2wingend]['slope'].values, 0, color='grey', linewidth = 0)
ax[0].set_ylim(-.1, 0.05)
ax[1].set_ylim(-.1, 0.05)
ax[2].set_ylim(-.1, 0.05)
fig.tight_layout(pad=3.0)
plt.show()

## $v_3$ band examination

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10.5,3))
ax[0].set_title("Raw v3 band")
ax[0].fill_between(series_averagedslopes[v3start:v3end].index,
                   series_averagedslopes[v3start:v3end].values, 0, color='grey', linewidth = 0)
ax[1].set_title("Interpolated v3 band")
ax[1].fill_between(slopes_df2[v3start:v3end].index, 
                   slopes_df2[v3start:v3end]['slope'].values, 0, color='grey', linewidth = 0)

markerline, stemlines, baseline = ax[1].stem(slopes_interp[v3start:v3end].index,
                                             slopes_interp[v3start:v3end]['slope'].values,
                                             markerfmt=' ', linefmt="k-", basefmt="grey", use_line_collection = True)
plt.setp(stemlines, 'linewidth', 0.25)
plt.setp(baseline, 'linewidth', 0)

ax[2].set_title("Negative-only v3 band")
ax[2].fill_between(slopes_df3[v3start:v3end].index,
                   slopes_df3[v3start:v3end]['slope'].values, 0, color='grey', linewidth = 0)
ax[0].set_ylim(-.0007, 0.00015)
ax[1].set_ylim(-.0007, 0.00015)
ax[2].set_ylim(-.0007, 0.00015)
fig.tight_layout(pad=3.0)
plt.show()

## Evaluate Integral (1)-(4)

In [ ]:
# Put series data into a dataframe so that we may evaluate integral with scipi.integrate.trapz:
rad_err_df = pd.DataFrame(index = series_averagederror.index, data = {'slope':series_averagederror.values})

# Evaluate window integral by Trapezoidal rule
v2_win = scipy.integrate.trapz(slopes_df3[win_start:win_end]['slope'].values, slopes_df3[win_start:win_end].index) * math.pi
v2_win_err = scipy.integrate.trapz(rad_err_df[win_start:win_end]['slope'].values, rad_err_df[win_start:win_end].index) * math.pi

# Evaluate v2 PQR integral by Trapezoidal rule
v2_PQR = scipy.integrate.trapz(slopes_df3[v2start:v2end]['slope'].values, slopes_df3[v2start:v2end].index) * math.pi
v2_PQR_err = scipy.integrate.trapz(rad_err_df[v2start:v2end]['slope'].values,
                                   rad_err_df[v2start:v2end].index) * math.pi
# Evaluate v2 PQR integral by Simpson's rule
v2_PQRS = scipy.integrate.simps(slopes_df3[v2start:v2end]['slope'].values, slopes_df3[v2start:v2end].index) * math.pi
v2_PQR_errS = scipy.integrate.simps(rad_err_df[v2start:v2end]['slope'].values, rad_err_df[v2start:v2end].index) * math.pi

# Evaluate v2 wing integral by Trapezoidal rule
v2_wing = scipy.integrate.trapz(slopes_df3[v2wingstart:v2wingend]['slope'].values,
                                slopes_df3[v2wingstart:v2wingend].index) * math.pi
v2_wing_err = scipy.integrate.trapz(rad_err_df[v2wingstart:v2wingend]['slope'].values,
                                    rad_err_df[v2wingstart:v2wingend].index) * math.pi
# Evaluate v2 wing integral by Simpson's rule
v2_wingS = scipy.integrate.simps(slopes_df3[v2wingstart:v2wingend]['slope'].values,
                                 slopes_df3[v2wingstart:v2wingend].index) * math.pi
v2_wing_errS = scipy.integrate.simps(rad_err_df[v2wingstart:v2wingend]['slope'].values,
                                     rad_err_df[v2wingstart:v2wingend].index) * math.pi

# Evaluate v3 integral by Trapezoidal rule
v3 = scipy.integrate.trapz(slopes_df3[v3start:v3end]['slope'].values, slopes_df3[v3start:v3end].index) * math.pi
v3_err = scipy.integrate.trapz(rad_err_df[v3start:v3end]['slope'].values, rad_err_df[v3start:v3end].index) * math.pi
# Evaluate v3 integral by Simpson's rule
v3S = scipy.integrate.simps(slopes_df3[v3start:v3end]['slope'].values, slopes_df3[v3start:v3end].index) * math.pi
v3_errS = scipy.integrate.simps(rad_err_df[v3start:v3end]['slope'].values, rad_err_df[v3start:v3end].index) * math.pi

# Quantify R-branch measurement gap
Interpolate_integral=scipy.integrate.trapz(slopes_df3[v2end-5.1:v2end]['slope'].values, 
                                           slopes_df3[v2end-5.1:v2end].index) * math.pi
Interpolate_int_err = scipy.integrate.trapz(rad_err_df[v2end-5.1:v2end]['slope'].values,
                                            rad_err_df[v2end-5.1:v2end].index) * math.pi

print('v2 win integral (Trapz) =','{:.2f}'.format(v2_win),'±','{:.2f}'.format(v2_win_err),'mW/m²/yr')
print('v2 PQR integral (Trapz) =','{:.2f}'.format(v2_PQR+Interpolate_integral),'±','{:.2f}'.format(v2_PQR_err+Interpolate_int_err),'mW/m²/yr')
print('v2 PQR integral (Simps) =','{:.2f}'.format(v2_PQRS+Interpolate_integral),'±','{:.2f}'.format(v2_PQR_errS+Interpolate_int_err),'mW/m²/yr')
print('v2 wing integral (Trapz) =','{:.2f}'.format(v2_wing),'±','{:.2f}'.format(v2_wing_err),'mW/m²/yr')
print('v2 wing integral (Simps) =','{:.2f}'.format(v2_wingS),'±','{:.2f}'.format(v2_wing_errS),'mW/m²/yr')
print('v3 integral     (Trapz) =','{:.2f}'.format(v3),'±','{:.2f}'.format(v3_err),'mW/m²/yr')
print('v3 integral     (Simps) =','{:.2f}'.format(v3S),'±','{:.2f}'.format(v3_errS),'mW/m²/yr')

AIRS_total =  (v2_PQR + 2 * v2_wing + Interpolate_integral + v3 + v2_win) * year_count
AIRS_err =  (v2_PQR_err + 2 * v2_wing_err  + v3_err + v2_win + Interpolate_int_err) * year_count

## Equations (1)-(4) and $\delta$M<sub>LW</sub>

In [ ]:
print('v2 PQR integral  =','{:.0f}'.format((v2_PQR+Interpolate_integral)*year_count),'±','{:.0f}'.format((v2_PQR_err+Interpolate_int_err)*year_count),'mW/m²/yr')
print('v2 wing integral =','{:.0f}'.format(v2_wing*year_count),'±','{:.0f}'.format(v2_wing_err*year_count),'mW/m²/yr')
print('v2 win integral  =','{:.1f}'.format(v2_win*year_count),'±','{:.1f}'.format(v2_win_err*year_count),'mW/m²/yr')
print('v3 integral      =','{:.0f}'.format(v3*year_count),'±','{:.0f}'.format(v3_err*year_count),'mW/m²/yr')
print("")
print('Total:', '{:.0f}'.format(AIRS_total), '±','{:.0f}'.format(AIRS_err),'mW/m²')

## CO<sub>2</sub> measurement

In [ ]:
# During AIRS observation period, atmospheric CO2 given by NOAA ESRL Global Division
# https://www.esrl.noaa.gov/gmd/ccgg/trends/gl_data.html
# start = 373.13 (Trend value September, 2002)
# end = 410.21 (Trend value Aug, 2019)

CO2_start = 373.13  # ppm CO2 where AIRS data begins
CO2_end = 410.21    # ppm CO2 where AIRS data ends

print("CO2 start:", CO2_start)
print("CO2 end:", CO2_end)
print("delta:",'{:.1f}'.format(CO2_end-CO2_start))

## Scaling function

In [ ]:
def F_MYHRE98(C, C0):
    return 5.35 * np.log(C / C0)

## Import CMIP6 4xCO<sub>2</sub> and scale down:

In [ ]:
CMIP6_ERF_LW_cs=pd.read_csv("./data/ERF_LW_cs_4xCO2.csv") # CMIP6 Data from Chris Smith
print(CMIP6_ERF_LW_cs)

## Table 1

In [ ]:
CMIP6 = CMIP6_ERF_LW_cs
CMIP6['M98x1.1'] = round(F_MYHRE98(CO2_end, CO2_start) / F_MYHRE98(4*278, 278) * CMIP6['ERF_LW_cs'], 3)
CMIP6['M98%AIRS'] = round((CMIP6['M98x1.1']/(-1* AIRS_total/1000)-1)*100, 0)
CMIP6 = CMIP6.drop(['Run'], axis=1)
CMIP6 = CMIP6.drop(CMIP6.index[[9, 12, 13, 14, 16]])
CMIP6 = CMIP6.reset_index(drop=True)
print(CMIP6)
CMIP6_avg = CMIP6['M98x1.1'].mean()
print('CMIP6 avg:', round(CMIP6_avg, 3),"   +", round((CMIP6_avg/(-1*AIRS_total/1000)-1)*100, 0),'% vs AIRS')

### What percent of CMIP6 CO<sub>2</sub> forcing is actually observed?

In [ ]:
print('AIRS:', '{:.0f}'.format(-1*AIRS_total), '±','{:.0f}'.format(AIRS_err),'mW/m²')
print('CMIP6:', '{:.0f}'.format(CMIP6_avg*1000) ,'mW/m²')
print('AIRS/CMIP6 =','{:.0f}'.format(AIRS_total / (CMIP6_avg*-1000)*100), '%')

## ----- END OF DATA PROCESSING ----

### To recreate Figure 1:
project a dotted box from +/- 60 lat, -130 to -190 long for Harries pseduo-global test area

### To recreate Figure 2:
plot the 'radiances' field for 'hdffile' at Track=0, xTrack=0 for AIRS.2016.01.05.004.L2.CC_IR.v6.0.31.0.G16005164855.hdf

### To recreate Figure 3:
Highlight areas of AIRS.2016.01.01.015.L1B.VIS_Rad.v5.0.0.0.G16001115936.hdf.jpg where Tot_CldCC4 = 0.

### To recreate Figure 4:
plot channels 982.838, 668.288, 740.973, 650.814 from OLRdata at -10to0 lat, 20to40 lon with stats.linregress on each series

### To recreate Figure 5:
Plot lat_rad_trends.index, lat_rad_trends for each latitude

### To recreate Figure 6:
Plot series_averagedslopes and shaded +/- series_averagederror

### To recreate Figure 7:
plot HITRAN lines between 450-900 cm-1 from spectralcalc.com

### To recreate Figure 8:
Plot 650-750 cm-1 from row 13 of granule 004 from Jan 5, 2016 (all radiances are relative to the average of (X-track(15)+X-track(16))/2